# Lesson 2 : motor and scan

In this lesson, we'll work with a motor (for precise positioning) in [Bluesky](http://nsls-ii.github.io/bluesky/) and [related tools](http://nsls-ii.github.io/).

First, we'll connect with an EPICS motor (using [ophyd](http://nsls-ii.github.io/ophyd/)), and then use the Bluesky software to scan the motor (with the scaler from [lesson 1](lesson1.ipynb)).

Load `ophyd` device support for the `EpicsMotor` and connect with one EPICS motor channel.  We have a synApps (v6.1) XXX-style IOC with the prefix `sky:`. It has a scaler, 16 soft channel motors, and some other support we'll ignore in this lesson.

**note**:  This tutorial expects to find an EPICS IOC on the local network configured as a synApps [`xxx`](https://github.com/epics-modules/xxx) IOC with prefix `sky:`.  A docker container is available to provide this IOC.  See this URL for instructions:  https://github.com/prjemian/epics-docker/blob/master/n3_synApps/README.md

In [1]:
from ophyd import EpicsMotor
m1 = EpicsMotor("sky:m1", name="m1")
m1.wait_for_connection()

Show the current value fo the motor (that's the `.RBV` field, in case you were interested).

In [2]:
m1.position

1.0

Connect the scaler (as was done in lesson 1).  Define some of the chaannel names and clear out others.

In [3]:
from ophyd.scaler import ScalerCH
scaler = ScalerCH("sky:scaler1", name="scaler")
scaler.wait_for_connection()

# Since there are no detectors actually connected to this scaler,
# we can change names at our choice.  A real scaler will have
# detectors connected to specific channels and we should not modify
# these names without regard to how signals are physically connected.
scaler.channels.chan01.chname.put("clock")
scaler.channels.chan02.chname.put("I0")
scaler.channels.chan03.chname.put("scint")
scaler.channels.chan04.chname.put("")
scaler.channels.chan05.chname.put("")
scaler.channels.chan06.chname.put("")
scaler.channels.chan07.chname.put("")
scaler.channels.chan08.chname.put("")
scaler.channels.chan09.chname.put("")

Use only the channels with EPICS names, those are the *interesting* channels.

In [4]:
scaler.select_channels(None)
scaler.read()

OrderedDict([('clock', {'value': 11000000.0, 'timestamp': 1616788492.864084}),
             ('I0', {'value': 7.0, 'timestamp': 1616788492.864084}),
             ('scint', {'value': 5.0, 'timestamp': 1616788492.864084}),
             ('ROI1', {'value': 0.0, 'timestamp': 1616788492.864084}),
             ('ROI2', {'value': 0.0, 'timestamp': 1616788492.864084}),
             ('scaler_time', {'value': 1.1, 'timestamp': 1616788492.864084})])

Create a RunEngine but do not connect it with a data collection strategy.  That will come in the next lessons.

In [5]:
from bluesky import RunEngine
import bluesky.plans as bp
RE = RunEngine({})

Run a step scan using the motor and the scaler.

In [6]:
RE(bp.scan([scaler], m1, -1, 1, 5))

('a1d2d2e7-2fd6-4e62-834c-2f1252c0059c',)

Ah, yes.  Nothing to see here since we did not setup anything to receive the *documents* from the RunEngine.  Here's the basic callback from lesson 1.

In [7]:
import pprint
def myCallback(key, doc):
    print()
    print(key, len(doc))
    pprint.pprint(doc)

Repeat the same scan but handle the document stream with `myCallback()`.

In [8]:
RE(bp.scan([scaler], m1, -1, 1, 5), myCallback)

                         'shape': [],
                                                          'source': 'PV:sky:scaler1.G2',
                                                          'units': None,
                                                          'upper_ctrl_limit': None}),
                                                        ('scaler_channels_chan03_chname',
                                                         {'dtype': 'string',
                                                          'lower_ctrl_limit': None,
                                                          'shape': [],
                                                          'source': 'PV:sky:scaler1.NM3',
                                                          'units': None,
                                                          'upper_ctrl_limit': None}),
                                                        ('scaler_channels_chan03_preset',
                                                     

('90f48d10-7583-4a77-991f-b78e76d1524f',)

# Summary

We'll show this code as a python program:

```
#!/usr/bin/env python

"lesson 2: motor"

from ophyd import EpicsMotor
from ophyd.scaler import ScalerCH
from bluesky import RunEngine
import bluesky.plans as bp
from apstools.devices import use_EPICS_scaler_channels


def myCallback(key, doc):
    print()
    print(key, len(doc))
    pprint.pprint(doc)


m1 = EpicsMotor("sky:m1", name="m1")
m1.wait_for_connection()
print(m1.position)

scaler = ScalerCH("sky:scaler1", name="scaler")
scaler.wait_for_connection()


# Since there are no detectors actually connected to this scaler,
# we can change names at our choice.  A real scaler will have
# detectors connected to specific channels and we should not modify
# these names without regard to how signals are physically connected.
scaler.channels.chan01.chname.put("clock")
scaler.channels.chan02.chname.put("I0")
scaler.channels.chan03.chname.put("scint")
scaler.channels.chan04.chname.put("")
scaler.channels.chan05.chname.put("")
scaler.channels.chan06.chname.put("")
scaler.channels.chan07.chname.put("")
scaler.channels.chan08.chname.put("")
scaler.channels.chan09.chname.put("")


scaler.match_names()
use_EPICS_scaler_channels(scaler)
print(scaler.read())

RE = RunEngine({})

RE(bp.scan([scaler], m1, -1, 1, 5))

RE(bp.scan([scaler], m1, -1, 1, 5), myCallback)
```